# Squat Tracker with PoseNet (ONNX Runtime)

This notebook uses **PoseNet** with **ONNX Runtime** - much lighter than TensorFlow! Should import quickly and provide accurate pose detection without kernel crashes.


In [1]:
# Import required libraries for YOLOv11 Pose
import cv2
import numpy as np
import math
import subprocess
import sys

# Install ultralytics in the kernel's Python
subprocess.check_call([sys.executable, "-m", "pip", "install", "ultralytics"])
from ultralytics import YOLO

print("Imports complete! Using YOLOv11 Pose!")


Imports complete! Using YOLOv11 Pose!


In [2]:
# YOLOv11 pose keypoint connections
POSE_CONNECTIONS = [
    # Face connections
    (0, 1), (0, 2), (1, 3), (2, 4),  # nose-eyes-ears
    
    # Upper body connections
    (5, 6),   # left shoulder - right shoulder
    (5, 7),   # left shoulder - left elbow
    (6, 8),   # right shoulder - right elbow
    (7, 9),   # left elbow - left wrist
    (8, 10),  # right elbow - right wrist
    
    # Torso connections
    (5, 11),  # left shoulder - left hip
    (6, 12),  # right shoulder - right hip
    (11, 12), # left hip - right hip
    
    # Lower body connections
    (11, 13), # left hip - left knee
    (12, 14), # right hip - right knee
    (13, 15), # left knee - left ankle
    (14, 16), # right knee - right ankle
]

def angle_between(v1, v2):
    """Calculate angle between two vectors in degrees"""
    dot = v1[0]*v2[0] + v1[1]*v2[1]
    mag1 = math.sqrt(v1[0]**2 + v1[1]**2)
    mag2 = math.sqrt(v2[0]**2 + v2[1]**2)
    if mag1 * mag2 == 0:
        return 0
    return math.degrees(math.acos(dot / (mag1 * mag2)))


In [3]:
# Load YOLOv11 Pose model
def load_yolo_pose():
    """Load YOLOv11 Pose model"""
    print("Loading YOLOv11 Pose model...")
    try:
        # Load YOLOv11 pose model (automatically downloads if not present)
        model = YOLO('yolo11n-pose.pt')  # nano version for speed
        print("YOLOv11 Pose loaded successfully!")
        return model
    except Exception as e:
        print(f"Error loading YOLOv11 Pose: {e}")
        raise e  # Don't fallback, just fail if YOLOv11 doesn't work

# Load the YOLOv11 Pose model
yolo_model = load_yolo_pose()


Loading YOLOv11 Pose model...
YOLOv11 Pose loaded successfully!


In [4]:
# YOLOv11 Pose processing functions
def detect_pose_yolo(model, frame):
    """Detect pose using YOLOv11 Pose"""
    try:
        # Run YOLOv11 pose detection
        results = model(frame, verbose=False)
        
        if results and len(results) > 0:
            result = results[0]
            
            # Check if keypoints are detected
            if result.keypoints is not None and len(result.keypoints.xy) > 0:
                # Get keypoints for the first person (highest confidence)
                keypoints = result.keypoints.xy[0].cpu().numpy()  # Shape: (17, 2)
                confidences = result.keypoints.conf[0].cpu().numpy()  # Shape: (17,)
                
                # Convert to our format: (x, y, confidence)
                keypoints_formatted = []
                for i, (kp, conf) in enumerate(zip(keypoints, confidences)):
                    x, y = kp
                    keypoints_formatted.append((float(x), float(y), float(conf)))
                
                return keypoints_formatted
        
        return None
    
    except Exception as e:
        print(f"YOLOv11 Pose inference error: {e}")
        return None


In [5]:
# Angle calculation and visualization functions
def calculate_knee_angles(keypoints):
    """Calculate knee angles for both legs using knee-centered vectors"""
    # Correct YOLOv11 keypoint indices for legs
    LEFT_HIP = 11      # Left Hip
    LEFT_KNEE = 13     # Left Knee  
    LEFT_ANKLE = 15    # Left Ankle
    RIGHT_HIP = 12     # Right Hip
    RIGHT_KNEE = 14    # Right Knee
    RIGHT_ANKLE = 16   # Right Ankle
    
    # Get keypoints for both legs
    left_hip = keypoints[LEFT_HIP]
    left_knee = keypoints[LEFT_KNEE]
    left_ankle = keypoints[LEFT_ANKLE]
    right_hip = keypoints[RIGHT_HIP]
    right_knee = keypoints[RIGHT_KNEE]
    right_ankle = keypoints[RIGHT_ANKLE]
    
    # Calculate left leg angle: vectors from knee (13->15) and (13->11)
    left_thigh = (left_hip[0] - left_knee[0], left_hip[1] - left_knee[1])    # 13->11 (knee to hip)
    left_shin = (left_ankle[0] - left_knee[0], left_ankle[1] - left_knee[1]) # 13->15 (knee to ankle)
    left_angle = angle_between(left_thigh, left_shin)
    
    # Calculate right leg angle: vectors from knee (14->12) and (14->16)
    right_thigh = (right_hip[0] - right_knee[0], right_hip[1] - right_knee[1])    # 14->12 (knee to hip)
    right_shin = (right_ankle[0] - right_knee[0], right_ankle[1] - right_knee[1]) # 14->16 (knee to ankle)
    right_angle = angle_between(right_thigh, right_shin)
    
    return left_angle, right_angle

def calculate_hip_angles(keypoints):
    """Calculate hip angles for both legs using hip-centered vectors"""
    # YOLOv11 keypoint indices
    LEFT_SHOULDER = 5   # Left Shoulder
    LEFT_HIP = 11       # Left Hip
    LEFT_KNEE = 13      # Left Knee
    RIGHT_SHOULDER = 6  # Right Shoulder
    RIGHT_HIP = 12      # Right Hip
    RIGHT_KNEE = 14     # Right Knee
    
    # Get keypoints
    left_shoulder = keypoints[LEFT_SHOULDER]
    left_hip = keypoints[LEFT_HIP]
    left_knee = keypoints[LEFT_KNEE]
    right_shoulder = keypoints[RIGHT_SHOULDER]
    right_hip = keypoints[RIGHT_HIP]
    right_knee = keypoints[RIGHT_KNEE]
    
    # Calculate left hip angle: vectors from hip (11->5) and (11->13)
    left_torso = (left_shoulder[0] - left_hip[0], left_shoulder[1] - left_hip[1])    # 11->5 (hip to shoulder)
    left_thigh = (left_knee[0] - left_hip[0], left_knee[1] - left_hip[1])            # 11->13 (hip to knee)
    left_angle = angle_between(left_torso, left_thigh)
    
    # Calculate right hip angle: vectors from hip (12->6) and (12->14)
    right_torso = (right_shoulder[0] - right_hip[0], right_shoulder[1] - right_hip[1])  # 12->6 (hip to shoulder)
    right_thigh = (right_knee[0] - right_hip[0], right_knee[1] - right_hip[1])          # 12->14 (hip to knee)
    right_angle = angle_between(right_torso, right_thigh)
    
    return left_angle, right_angle

def draw_pose(frame, keypoints):
    """Draw pose on frame using correct YOLOv11 connections"""
    # Draw keypoints
    for i, (x, y, conf) in enumerate(keypoints):
        if conf > 0.3:  # Only draw confident keypoints
            cv2.circle(frame, (int(x), int(y)), 5, (0, 255, 0), -1)
            cv2.putText(frame, str(i), (int(x), int(y) - 10), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.3, (255, 255, 255), 1)
    
    # Draw connections using YOLOv11 pose connections
    for connection in POSE_CONNECTIONS:
        start_idx, end_idx = connection
        if start_idx < len(keypoints) and end_idx < len(keypoints):
            start_point = keypoints[start_idx]
            end_point = keypoints[end_idx]
            
            if start_point[2] > 0.3 and end_point[2] > 0.3:
                cv2.line(frame, 
                        (int(start_point[0]), int(start_point[1])),
                        (int(end_point[0]), int(end_point[1])),
                        (255, 0, 0), 2)


In [6]:
# Load video
video_path = "squat_video.MOV"  
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print(f"Error: Could not open video {video_path}")
else:
    # Get video properties
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    
    print(f"Video loaded: {width}x{height} at {fps} FPS")
    print("Ready to start Squat Tracker processing!")


Video loaded: 1080x1920 at 58.80863855707618 FPS
Ready to start Squat Tracker processing!


In [7]:
# UPDATED MAIN PROCESSING LOOP WITH VIDEO TIME TRACKING
# Initialize tracking variables
rep_count = 0
phase = "standing"  # "standing", "squatting"
frame_count = 0
SQUAT_DOWN_THRESHOLD = 130  # Start squatting when angle goes below this
SQUAT_UP_THRESHOLD = 150    # Stop squatting when angle goes above this
lowest_left_knee_angle_this_rep = None   # Track lowest left knee angle during current squat
lowest_right_knee_angle_this_rep = None  # Track lowest right knee angle during current squat
lowest_left_hip_angle_this_rep = None    # Track lowest left hip angle during current squat
lowest_right_hip_angle_this_rep = None   # Track lowest right hip angle during current squat
rep_history = []  # Store all completed reps with their lowest angles and durations
squat_start_frame = None  # Track when squat started

print("Starting Squat Tracker processing...")
print("Tracking both leg knee angles for squat detection...")
print("Tracking both leg hip angles for additional analysis...")
print(f"Squat down threshold: {SQUAT_DOWN_THRESHOLD}°")
print(f"Squat up threshold: {SQUAT_UP_THRESHOLD}°")
print("Phase tracking: standing → squatting → standing (count rep)")
print("Tracks lowest depth angle for each rep")
print("Uses hysteresis to prevent threshold bouncing")

# Main processing loop with YOLOv11 Pose - UPDATED VERSION WITH HIP ANGLES AND VIDEO TIME
print("Using YOLOv11 Pose for squat tracking!")

# Setup video writer for output - using MJPG codec for QuickTime compatibility
output_path = "squat_tracker_output.mov"
fourcc = cv2.VideoWriter_fourcc(*'MJPG')
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

print(f"Saving processed video to: {output_path}")

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    frame_count += 1
    
    # Detect pose using YOLOv11
    keypoints = detect_pose_yolo(yolo_model, frame)
    
    if keypoints:
        # Calculate knee angles for both legs
        left_knee_angle, right_knee_angle = calculate_knee_angles(keypoints)
        
        # Calculate hip angles for both legs
        left_hip_angle, right_hip_angle = calculate_hip_angles(keypoints)
        
        if left_knee_angle is not None and right_knee_angle is not None:
            # Check if either leg angle crosses thresholds (hysteresis)
            min_knee_angle = min(left_knee_angle, right_knee_angle)
            
            if phase == "standing" and min_knee_angle < SQUAT_DOWN_THRESHOLD:
                phase = "squatting"
                lowest_left_knee_angle_this_rep = left_knee_angle   # Initialize lowest angles
                lowest_right_knee_angle_this_rep = right_knee_angle
                lowest_left_hip_angle_this_rep = left_hip_angle if left_hip_angle is not None else None
                lowest_right_hip_angle_this_rep = right_hip_angle if right_hip_angle is not None else None
                squat_start_frame = frame_count  # Start timing
                print(f"Frame {frame_count}: Started SQUATTING (knee angle: {min_knee_angle:.1f})")
            
            elif phase == "squatting":
                # Update lowest knee angles if current angles are smaller
                if left_knee_angle < lowest_left_knee_angle_this_rep:
                    lowest_left_knee_angle_this_rep = left_knee_angle
                if right_knee_angle < lowest_right_knee_angle_this_rep:
                    lowest_right_knee_angle_this_rep = right_knee_angle
                
                # Update lowest hip angles if current angles are smaller
                if left_hip_angle is not None and (lowest_left_hip_angle_this_rep is None or left_hip_angle < lowest_left_hip_angle_this_rep):
                    lowest_left_hip_angle_this_rep = left_hip_angle
                if right_hip_angle is not None and (lowest_right_hip_angle_this_rep is None or right_hip_angle < lowest_right_hip_angle_this_rep):
                    lowest_right_hip_angle_this_rep = right_hip_angle
                
                # Check if back to standing (using higher threshold)
                if min_knee_angle >= SQUAT_UP_THRESHOLD:
                    phase = "standing"
                    rep_count += 1
                    squat_duration_frames = frame_count - squat_start_frame
                    squat_duration_seconds = squat_duration_frames / fps
                    rep_history.append((rep_count, lowest_left_knee_angle_this_rep, lowest_right_knee_angle_this_rep, 
                                      lowest_left_hip_angle_this_rep, lowest_right_hip_angle_this_rep, squat_duration_seconds))
                    print(f"Rep {rep_count}: squat completed")
                    print(f"  Left Knee: {lowest_left_knee_angle_this_rep:.1f} deg, Right Knee: {lowest_right_knee_angle_this_rep:.1f} deg")
                    print(f"  Left Hip: {lowest_left_hip_angle_this_rep:.1f} deg, Right Hip: {lowest_right_hip_angle_this_rep:.1f} deg")
                    print(f"  Duration: {squat_duration_seconds:.2f}s")
                    # Reset for next rep
                    lowest_left_knee_angle_this_rep = None
                    lowest_right_knee_angle_this_rep = None
                    lowest_left_hip_angle_this_rep = None
                    lowest_right_hip_angle_this_rep = None
                    squat_start_frame = None
            
            # Draw pose
            draw_pose(frame, keypoints)
            
            # Draw angle info on left side with larger fonts
            cv2.putText(frame, f'Left Knee Angle: {left_knee_angle:.1f} deg', 
                       (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 255, 0), 2)
            cv2.putText(frame, f'Right Knee Angle: {right_knee_angle:.1f} deg', 
                       (50, 80), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 255, 0), 2)
            
            # Draw hip angle info
            if left_hip_angle is not None and right_hip_angle is not None:
                cv2.putText(frame, f'Left Hip Angle: {left_hip_angle:.1f} deg', 
                           (50, 110), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 255, 255), 2)
                cv2.putText(frame, f'Right Hip Angle: {right_hip_angle:.1f} deg', 
                           (50, 140), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 255, 255), 2)
            
            # Show lowest angles if currently squatting
            if phase == "squatting" and lowest_left_knee_angle_this_rep is not None:
                cv2.putText(frame, f'Lowest L-Knee: {lowest_left_knee_angle_this_rep:.1f} deg', 
                           (50, 170), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (255, 0, 255), 2)
                cv2.putText(frame, f'Lowest R-Knee: {lowest_right_knee_angle_this_rep:.1f} deg', 
                           (50, 200), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (255, 0, 255), 2)
            
            if phase == "squatting" and lowest_left_hip_angle_this_rep is not None:
                cv2.putText(frame, f'Lowest L-Hip: {lowest_left_hip_angle_this_rep:.1f} deg', 
                           (50, 230), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (255, 0, 255), 2)
                cv2.putText(frame, f'Lowest R-Hip: {lowest_right_hip_angle_this_rep:.1f} deg', 
                           (50, 260), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (255, 0, 255), 2)
            
            # Show current squat duration if squatting (LIVE TIMER)
            if phase == "squatting" and squat_start_frame is not None:
                current_duration = (frame_count - squat_start_frame) / fps
                cv2.putText(frame, f'Squat Time: {current_duration:.1f}s', 
                           (50, 290), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (255, 255, 255), 2)
            
            # Show thresholds (removed - no longer displaying threshold text)
            
            # Draw rep history on right side with organized display
            right_x = width - 450  # Start 450 pixels from right edge (wider for more info)
            y_start = 50
            cv2.putText(frame, "Rep History:", (right_x, y_start), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (255, 255, 255), 2)
            
            for i, (rep_num, lowest_left_knee, lowest_right_knee, lowest_left_hip, lowest_right_hip, duration) in enumerate(rep_history):
                y_pos = y_start + 30 + (i * 150)  # Increased spacing to 150 pixels between reps for better readability
                cv2.putText(frame, f"Rep {rep_num}:", (right_x, y_pos), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
                cv2.putText(frame, f"  L-Knee: {lowest_left_knee:.1f} deg", (right_x, y_pos + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
                cv2.putText(frame, f"  R-Knee: {lowest_right_knee:.1f} deg", (right_x, y_pos + 40), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
                cv2.putText(frame, f"  L-Hip: {lowest_left_hip:.1f} deg", (right_x, y_pos + 60), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)
                cv2.putText(frame, f"  R-Hip: {lowest_right_hip:.1f} deg", (right_x, y_pos + 80), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)
                cv2.putText(frame, f"  Duration: {duration:.1f}s", (right_x, y_pos + 100), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
    
    # Display info with larger fonts
    cv2.putText(frame, f'Squats: {rep_count}', (50, 350), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (255, 0, 0), 2)
    cv2.putText(frame, f'Phase: {phase}', (50, 390), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 255, 0), 2)
    cv2.putText(frame, f'Method: YOLOv11 Pose', (50, 430), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (255, 255, 0), 2)
    
    # Draw live video time at bottom right with proper margins
    video_time_seconds = frame_count / fps
    # Calculate text size to ensure proper positioning
    font_scale = 1.0
    font_thickness = 2
    text_size_time = cv2.getTextSize(f"Video Time: {video_time_seconds:.1f}s", cv2.FONT_HERSHEY_SIMPLEX, font_scale, font_thickness)[0]
    text_size_frame = cv2.getTextSize(f"Frame: {frame_count}", cv2.FONT_HERSHEY_SIMPLEX, font_scale, font_thickness)[0]
    
    # Position text with proper margins (at least 20 pixels from edges)
    margin_right = max(text_size_time[0], text_size_frame[0]) + 20
    margin_bottom = 50  # Increased margin from bottom
    
    cv2.putText(frame, f"Video Time: {video_time_seconds:.1f}s", 
               (width - margin_right, height - margin_bottom), cv2.FONT_HERSHEY_SIMPLEX, font_scale, (255, 255, 255), font_thickness)
    cv2.putText(frame, f"Frame: {frame_count}", 
               (width - margin_right, height - margin_bottom + 30), cv2.FONT_HERSHEY_SIMPLEX, font_scale, (255, 255, 255), font_thickness)
    
    # Write frame to output video
    out.write(frame)
    
    # Print progress every 100 frames
    if frame_count % 100 == 0:
        print(f"Processed {frame_count} frames...")

print(f"Final squat count: {rep_count}")
print(f"Video saved to: {output_path}")


Starting Squat Tracker processing...
Tracking both leg knee angles for squat detection...
Tracking both leg hip angles for additional analysis...
Squat down threshold: 130°
Squat up threshold: 150°
Phase tracking: standing → squatting → standing (count rep)
Tracks lowest depth angle for each rep
Uses hysteresis to prevent threshold bouncing
Using YOLOv11 Pose for squat tracking!
Saving processed video to: squat_tracker_output.mov


OpenCV: FFMPEG: tag 0x47504a4d/'MJPG' is not supported with codec id 7 and format 'mov / QuickTime / MOV'
OpenCV: FFMPEG: fallback to use tag 0x6765706a/'jpeg'
[W NNPACK.cpp:64] Could not initialize NNPACK! Reason: Unsupported hardware.


Frame 72: Started SQUATTING (knee angle: 128.8)
Processed 100 frames...
Processed 200 frames...
Rep 1: squat completed
  Left Knee: 72.3 deg, Right Knee: 64.8 deg
  Left Hip: 68.6 deg, Right Hip: 78.0 deg
  Duration: 2.60s
Processed 300 frames...
Processed 400 frames...
Frame 411: Started SQUATTING (knee angle: 127.5)
Processed 500 frames...
Rep 2: squat completed
  Left Knee: 75.0 deg, Right Knee: 69.5 deg
  Left Hip: 68.3 deg, Right Hip: 77.5 deg
  Duration: 2.62s
Processed 600 frames...
Processed 700 frames...
Processed 800 frames...
Frame 895: Started SQUATTING (knee angle: 127.3)
Processed 900 frames...
Processed 1000 frames...
Rep 3: squat completed
  Left Knee: 72.9 deg, Right Knee: 67.7 deg
  Left Hip: 67.0 deg, Right Hip: 75.9 deg
  Duration: 3.06s
Processed 1100 frames...
Processed 1200 frames...
Final squat count: 3
Video saved to: squat_tracker_output.mov


In [8]:
# Cleanup
cap.release()
out.release()  # Release video writer
cv2.destroyAllWindows()
print("Processing complete!")


Processing complete!
